In [138]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [139]:
ps5URL = 'https://www.nowinstock.net/videogaming/consoles/sonyps5/full_history.php'
ps5Page = requests.get(ps5URL)

In [140]:
soup = BeautifulSoup(ps5Page.content, 'html.parser')

In [141]:
table = soup.find_all('table')[0]

In [142]:
date = []
status = []
count = 0
for row in table.find_all('tr'):
    columnCount = 0
    columns = table.find_all('td')
    for column in columns:
        if count % 2 == 0:
            date.append(column.get_text())
        else: 
            status.append(column.get_text())
        count += 1
        
cleanedList = [date, status]

In [149]:
dirtyDF = pd.DataFrame(cleanedList[0], cleanedList[1]).drop_duplicates().reset_index()
dirtyDF[['date','time']] = dirtyDF[0].str.split(' - ', expand = True)
dirtyDF[['store','info']] = dirtyDF['index'].str.split(' - ', expand = True)

dirtyDF['date'] = pd.to_datetime(dirtyDF['date'], infer_datetime_format=True)
dirtyDF['weekday'] = dirtyDF['date'].dt.day_name()

dirtyDF['time'] = pd.to_datetime(dirtyDF['time'], infer_datetime_format=True)
dirtyDF['time'] = dirtyDF['time'].dt.time

dirtyDF['event'] = dirtyDF['info'].apply(lambda x: 'In Stock'
                                    if 'In Stock' in x
                                    else 'Out of Stock')

dirtyDF['consoleType'] = dirtyDF['info'].apply(lambda x: 'Digital'
                                              if 'Digital' in x
                                              else 'Digital + Disc')

stockDF = dirtyDF.drop(columns=['index',0,'info']) # produce stockDF, cleaned dataset of in/out stock events

/Users/jonathan/Library/Python/3.8/lib/python/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [155]:
# what stores are we tracking?
stockDF['store'].unique()

array(['Playstation Direct', 'Best Buy', 'Costco', 'Target', 'GameStop',
       'Walmart', 'Amazon', "Sam's Club", 'antOnline', 'Dell Home',
       'Curacao.com', 'B&H Photo'], dtype=object)

In [154]:
stockDF[stockDF['event']=='In Stock'].groupby(['store','weekday']).count()

date  time  event  consoleType
store              weekday                                  
Amazon             Friday        1     1      1            1
                   Thursday      4     4      4            4
                   Tuesday       4     4      4            4
                   Wednesday     3     3      3            3
Best Buy           Friday       19    19     19           19
                   Monday        1     1      1            1
                   Sunday        1     1      1            1
                   Thursday      7     7      7            7
Costco             Friday        2     2      2            2
                   Thursday      3     3      3            3
                   Tuesday       1     1      1            1
GameStop           Friday        6     6      6            6
                   Monday       10    10     10           10
                   Saturday      1     1      1            1
                   Thursday      3     3      3            3
Playstation Direct Friday       14    14     14           14
                   Monday        7     7      7            7
                   Thursday     15    15     15           15
                   Tuesday       9     9      9            9
                   Wednesday    15    15     15           15
Sam's Club         Thursday      1     1      1            1
                   Tuesday       1     1      1            1
                   Wednesday     2     2      2            2
Target             Friday        1     1      1            1
                   Monday        1     1      1            1
                   Thursday     11    11     11           11
                   Tuesday       7     7      7            7
                   Wednesday     4     4      4            4
Walmart            Thursday      7     7      7            7
                   Tuesday       5     5      5            5

In [ ]:
# To Do: pull indicators out of info (digital/normal, in stock, out of stock), then think about how to tell when stock drops happen